# Milestone 4

## <b>Part 1:</b> 
#### Convert your previously written Beam pipeline(s) to Dataflow. Run them on Dataflow over the entire input data; debug and fix as necessary.

### Employer_Dataflow
#### We removed duplicates in Employer table

In [11]:
%run employer_dataflow.py

### Application_Dataflow
#### We removed duplicates in Application table and then JOINed it with Employer table ON employer_id. 

In [11]:
%run application_dataflow.py

### Ownership_Dataflow
#### We removed duplicates in Ownership table. Then we created unique ownership_id just for the simplicity sake. 

In [13]:
%run ownership_dataflow.py

### Occupation_Dataflow

#### We decided to create unique job_id column for simplicity sake. So we copied the regular Occupation table and created unique job_id through BQ, then processed Dataflow transformation.
#### In addition, there were different formats like "15/1121", "13.111", "1-1132", "19-104" and "2911123" other than those formats we transformed in Milestone3. So we fixed it
#### We decided to drop employer_citym employer_country, employment_start_date and employment_end_date since our goal is to analyze these datasets and hope to gain knowledge about which areas of occupation, or occupation itself, have requested for many H-1B.

In [6]:
%run occupation_dataflow.py

In [1]:
!bq rm -f -t H_1B_refined.Occupation_Dataflow

## <b>Part 2:</b> 
#### Verify that each BigQuery output table contains a valid primary key. Child tables must also have a valid foreign key. Run the appropriate SQL statements within your milestone4 notebook to verify these constraints.

### Employer_Dataflow

In [1]:
%%bigquery
SELECT employer_id, COUNT(employer_id) AS count
FROM H_1B_refined.Employer_Dataflow
GROUP BY employer_id
HAVING count > 1

,employer_id,count


### Application_Dataflow
#### Verify PK

In [2]:
%%bigquery
SELECT case_id, COUNT(case_id) AS count
FROM H_1B_refined.Application_Dataflow
GROUP BY case_id
HAVING count > 1

,case_id,count


#### Verify FK (FK is working fine)

In [4]:
%%bigquery
SELECT app.case_id, app.employer_id, emp.employer_name 
FROM H_1B_refined.Application_Dataflow AS app
JOIN H_1B_refined.Employer_Dataflow AS emp ON app.employer_id = emp.employer_id
LIMIT 10

,case_id,employer_id,employer_name
0,-7472653947438924331,7830068956980211394,APPLE INC.
1,3302131368530300568,3549615032242082539,AMAZON CORPORATE LLC
2,8351805987742448133,-3994438770359546112,MOLEX INCORPORATED
3,-6974557907243539782,9029377741145082496,GOOGLE INC.
4,6882660618389160379,-6616637527994182381,JPMORGAN CHASE & CO.
5,-6806937993996666628,-3081803327307139476,WS ATKINS INC.
6,-1173881495870676081,-7639904003258062437,"WAL-MART ASSOCIATES, INC."
7,-2095625144286580005,-6287398949312563689,"SONUSLAND, INC."
8,4804839157779805525,1099895834805669366,"CONILL ADVERTISING, INC."
9,-6680961539126776621,-9176787642659334798,BANFI PRODUCTS CORPORATION


### Ownership_Dataflow

In [7]:
%%bigquery
SELECT ownership_id, occ_code, COUNT(ownership_id) AS ownership_id_count, COUNT(occ_code) AS occ_code_count
FROM H_1B_refined.Ownership_Dataflow
GROUP BY ownership_id, occ_code
HAVING ownership_id_count > 1 AND occ_code_count > 1

,ownership_id,occ_code,ownership_id_count,occ_code_count


### Occupation_Dataflow
#### Verify PK

In [1]:
%%bigquery
SELECT job_id, COUNT(job_id) AS count
FROM H_1B_refined.Occupation_Dataflow
GROUP BY job_id
HAVING count > 1

,job_id,count


#### Verify FK

In [2]:
%%bigquery
SELECT occ.job_id, occ.employer_id, emp.employer_name 
FROM H_1B_refined.Occupation_Dataflow AS occ
JOIN H_1B_refined.Employer_Dataflow AS emp ON occ.employer_id = emp.employer_id
LIMIT 10

,job_id,employer_id,employer_name
0,7589577605519328432,6836890735342976109,CASE WESTERN RESERVE UNIVERSITY
1,-7264933987445979269,7179208282451432363,THE CURATORS OF THE UNIVERSITY OF MISSOURI
2,1526024896656132888,-3682059893006365487,UNIVERSITY OF ARKANSAS FOR MEDICAL SCIENCES
3,558701564645987523,7543754983785773736,"MICRON TECHNOLOGY, INC."
4,438983941118342196,-4637454601404055829,FEI YUE PRESCHOOL CORPORATION
5,-7929807996375999978,-1455827041719347063,UNIVERSITY OF MASSACHUSETTS AMHERST
6,-8637961728406129460,2459277965191890710,"COGENT HEALTHCARE OF MONTANA, P.C."
7,4285546324415181731,-980518890748306492,UNIVERSITY OF TEXAS RIO GRANDE VALLEY
8,-1821303640500437538,9212407996069603420,TEXAS A&M UNIVERSITY
9,4720791492937511805,-3492581103543470874,WALGREEN CO.


## <b>Part 3:</b>
### 1. Implement your cross-dataset queries:

#### Find job titles that are certified and count them. Since the table is huge, narrow it down to those with more than 1000000000. Then order it by count in descending order.

In [6]:
%%bigquery
SELECT job_title, COUNT(job_title) as count
FROM H_1B_refined.Occupation_Dataflow as occ
JOIN H_1B_refined.Employer_Dataflow as emp ON emp.employer_id = occ.employer_id
JOIN H_1B_refined.Application_Dataflow as app ON app.employer_id = emp.employer_id
WHERE app.CASE_STATUS = "CERTIFIED"
GROUP BY job_title
HAVING count > 1000000000
ORDER BY count DESC

,job_title,count
0,DEVELOPER,4810709587
1,SENIOR CONSULTANT,3695455903
2,CONSULTANT,3375268490
3,DEVELOPER USER INTERFACE,3371066803
4,TECHNOLOGY LEAD - US,2347395997
5,MANAGER,1837976827
6,ARCHITECT,1494966888
7,ANALYST,1323508956
8,TECHNOLOGY ANALYST - US,1320229033


#### Find how long does H-1B process take. We wanted to compare between certified (including CERTIFIED-WITHDRAWN) and denied. We grouped them by  its case status and took average of its date difference. We also put the query without averaging the result for additional information in Data Studio part.

In [1]:
%%bigquery
SELECT CASE_STATUS, AVG(DateDiff) as DateDiff_avg
FROM
(SELECT emp.employer_name, CASE_STATUS, CASE_SUBMITTED, DECISION_DATE, ABS(DATE_DIFF(CASE_SUBMITTED, DECISION_DATE, day)) AS DateDiff
FROM H_1B_refined.Application_Dataflow as app
JOIN H_1B_refined.Employer_Dataflow as emp ON emp.employer_id = app.employer_id
WHERE CASE_STATUS LIKE "%CERTIFIED%" OR CASE_STATUS = "DENIED")
GROUP BY CASE_STATUS


,CASE_STATUS,DateDiff_avg
0,CERTIFIED-WITHDRAWN,314.607819
1,CERTIFIED,5.861993
2,DENIED,4.459632


In [3]:
%%bigquery
SELECT emp.employer_name, CASE_STATUS, CASE_SUBMITTED, DECISION_DATE, ABS(DATE_DIFF(CASE_SUBMITTED, DECISION_DATE, day)) AS DateDiff
FROM H_1B_refined.Application_Dataflow as app
JOIN H_1B_refined.Employer_Dataflow as emp ON emp.employer_id = app.employer_id
WHERE CASE_STATUS LIKE "%CERTIFIED%" OR CASE_STATUS = "DENIED"

,employer_name,CASE_STATUS,CASE_SUBMITTED,DECISION_DATE,DateDiff
0,APPLE INC.,CERTIFIED-WITHDRAWN,2018-03-05,2018-04-19,45
1,AMAZON CORPORATE LLC,CERTIFIED-WITHDRAWN,2014-04-22,2015-02-17,301
2,MOLEX INCORPORATED,CERTIFIED-WITHDRAWN,2015-03-23,2015-04-22,30
3,GOOGLE INC.,CERTIFIED-WITHDRAWN,2016-08-11,2016-08-24,13
4,JPMORGAN CHASE & CO.,CERTIFIED-WITHDRAWN,2016-02-10,2016-11-10,274
...,...,...,...,...,...
5045689,ALSCO INC.,CERTIFIED-WITHDRAWN,2018-03-06,2018-03-15,9
5045690,ALSCO INC.,CERTIFIED-WITHDRAWN,2018-03-06,2018-03-15,9
5045691,KOHN PEDERSEN FOX ASSOCIATES PC,CERTIFIED-WITHDRAWN,2018-03-06,2018-03-15,9
5045692,"OMEGACUBE TECHNOLOGIES, INC.",CERTIFIED-WITHDRAWN,2018-03-06,2018-03-15,9


#### Here, we are trying to see which ownership (private or local government, etc..) of occupation tends to request for H-1B petition more.

In [13]:
%%bigquery
SELECT DISTINCT ownership, COUNT(ownership) as count
FROM H_1B_refined.Ownership_Dataflow as own
JOIN H_1B_refined.Occupation_Dataflow as occ ON occ.soc_code = own.occ_code
JOIN H_1B_refined.Employer_Dataflow as emp ON emp.employer_id = occ.employer_id
WHERE emp.h_1b_dependent = true 
GROUP BY ownership

,ownership,count
0,Private,76520739
1,Local government,76167213
2,State government,76097963
3,"Federal, State, and Local government",76488575
4,Federal government,65656075


#### We are trying to see any income difference between those areas/occupations that companies tend to request for H-1B.

In [6]:
%%bigquery
SELECT employer_name, occ.soc_title, occ.prevailing_wage_YR
FROM H_1B_refined.Employer_Dataflow as emp
JOIN H_1B_refined.Occupation_Dataflow as occ ON occ.employer_id = emp.employer_id
WHERE emp.h_1b_dependent = true 
ORDER BY occ.prevailing_wage_YR DESC
LIMIT 20

,employer_name,soc_title,prevailing_wage_YR
0,"I&I SOFTWARE, INC.","SOFTWARE DEVELOPERS, APPLICATIONS",9110400.0
1,RGB SYSTEMS,COMPUTER SYSTEMS ANALYSTS,784163.0
2,"TECH MAHINDRA (AMERICAS), INC.",COMPUTER SYSTEMS ANALYSTS,743810.0
3,ALTIMETRIK CORP,"SOFTWARE DEVELOPERS, APPLICATIONS",743590.0
4,ALTIMETRIK CORP,"SOFTWARE DEVELOPERS, APPLICATIONS",743590.0
5,PROMATRIX CORP,"SOFTWARE DEVELOPERS, APPLICATIONS",731240.0
6,AZEE SYSTEMS INC,"SOFTWARE DEVELOPERS, APPLICATIONS",671840.0
7,AZEE SYSTEMS INC,"SOFTWARE DEVELOPERS, APPLICATIONS",671840.0
8,AZEE SYSTEMS INC,"SOFTWARE DEVELOPERS, APPLICATIONS",671840.0
9,"HCL AMERICA, INC.",COMPUTER PROGRAMMERS,627541.0


### Wrap each query into a view and insert them into insight dataset

In [2]:
%%bigquery
CREATE VIEW insights.v_date_diff_by_case_status as 
SELECT CASE_STATUS, AVG(DateDiff) as DateDiff_avg
FROM
(SELECT emp.employer_name, CASE_STATUS, CASE_SUBMITTED, DECISION_DATE, ABS(DATE_DIFF(CASE_SUBMITTED, DECISION_DATE, day)) AS DateDiff
FROM H_1B_refined.Application_Dataflow as app
JOIN H_1B_refined.Employer_Dataflow as emp ON emp.employer_id = app.employer_id
WHERE CASE_STATUS LIKE "%CERTIFIED%" OR CASE_STATUS = "DENIED")
GROUP BY CASE_STATUS

""


In [5]:
%%bigquery
CREATE VIEW insights.v_date_diff_by_case_status_raw as 
SELECT emp.employer_name, CASE_STATUS, CASE_SUBMITTED, DECISION_DATE, ABS(DATE_DIFF(CASE_SUBMITTED, DECISION_DATE, day)) AS DateDiff
FROM H_1B_refined.Application_Dataflow as app
JOIN H_1B_refined.Employer_Dataflow as emp ON emp.employer_id = app.employer_id
WHERE CASE_STATUS LIKE "%CERTIFIED%" OR CASE_STATUS = "DENIED"

""


In [8]:
%%bigquery
CREATE VIEW insights.v_ownership_count as 
SELECT DISTINCT ownership, COUNT(ownership) as count
FROM H_1B_refined.Ownership_Dataflow as own
JOIN H_1B_refined.Occupation_Dataflow as occ ON occ.soc_code = own.occ_code
JOIN H_1B_refined.Employer_Dataflow as emp ON emp.employer_id = occ.employer_id
WHERE emp.h_1b_dependent = true 
GROUP BY ownership
LIMIT 100

""


In [5]:
%%bigquery
CREATE VIEW insights.v_prevailing_wage as 
SELECT employer_name, occ.soc_title, occ.prevailing_wage_YR
FROM H_1B_refined.Employer_Dataflow as emp
JOIN H_1B_refined.Occupation_Dataflow as occ ON occ.employer_id = emp.employer_id
WHERE emp.h_1b_dependent = true 
ORDER BY occ.prevailing_wage_YR DESC
LIMIT 100

""
